In [7]:
#TODO: make this nicer
path = "data\\tox21_original\\tox21.sdf"

from rdkit import Chem
import pandas as pd
data_molecules = Chem.SDMolSupplier(path)

info_file = pd.read_csv("data\\tox21_original\\tox21_compoundData.csv", sep=",", header=0)

targets = info_file.to_numpy()[:, -12:]
ids = info_file["ID"].to_numpy()
to_del = []
with open("file.csv", mode = "w") as fi:
    fi.write("NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles\n")

    for mol, target, i, infofile in zip(data_molecules, targets, ids, range(len(info_file))):
        try:
            smiles = Chem.MolToSmiles(mol)
            #fi.write(",".join([str(int(t)) if str(t)!="nan" else "" for t in target]) + "," + i + "," + smiles + "\n")
        except BaseException:
            to_del.append(infofile)
